In [2]:
from utils.augmentations import FastBaseTransform
import cv2
import numpy as np
import torch
import onnxruntime as rt
import time

# Load model
sess = rt.InferenceSession("yolact.onnx")
input_name = sess.get_inputs()[0].name
loc_name = sess.get_outputs()[0].name
conf_name = sess.get_outputs()[1].name
mask_name = sess.get_outputs()[2].name
priors_name = sess.get_outputs()[3].name
proto_name = sess.get_outputs()[4].name

In [3]:
input_file_path = "/media/ernestas/D1/workdir/yolact/data/rand.jpg"
frame = torch.from_numpy(cv2.imread(input_file_path)).float()
batch = FastBaseTransform()(frame.unsqueeze(0))
# inference
pred_onx = sess.run([loc_name, conf_name, mask_name, priors_name, proto_name], {input_name: batch.cpu().detach().numpy()})

In [4]:
for i,pred in enumerate(pred_onx):
    np.save(str(i), pred)

In [5]:
# import time
# start = time.time()
# for i in range(25):
#     pred_onx = sess.run([loc_name, conf_name, mask_name, priors_name, proto_name], {input_name: batch.cpu().detach().numpy()})
#     print(f"Average inference time: {(time.time()-start)/(i+1)}")

In [10]:
from layers import Detect
from postutils import *

detect = Detect(81, bkg_label=0, top_k=200, conf_thresh=0.05, nms_thresh=0.5)
preds = detect({'loc': torch.from_numpy(pred_onx[0]), 
                'conf': torch.from_numpy(pred_onx[1]), 
                'mask': torch.from_numpy(pred_onx[2]), 
                'priors': torch.from_numpy(pred_onx[3]),
                'proto': torch.from_numpy(pred_onx[4])}, None)

from importlib import reload  
import postutils
reload(postutils)
import matplotlib.pyplot as plt
img_numpy = prep_display(preds, frame, None, None, undo_transform=False)
plt.imshow(img_numpy)
plt.show()

RuntimeError: Error(s) in loading state_dict for Yolact:
	Missing key(s) in state_dict: "backbone.layers.2.6.conv1.weight", "backbone.layers.2.6.bn1.weight", "backbone.layers.2.6.bn1.bias", "backbone.layers.2.6.bn1.running_mean", "backbone.layers.2.6.bn1.running_var", "backbone.layers.2.6.conv2.weight", "backbone.layers.2.6.bn2.weight", "backbone.layers.2.6.bn2.bias", "backbone.layers.2.6.bn2.running_mean", "backbone.layers.2.6.bn2.running_var", "backbone.layers.2.6.conv3.weight", "backbone.layers.2.6.bn3.weight", "backbone.layers.2.6.bn3.bias", "backbone.layers.2.6.bn3.running_mean", "backbone.layers.2.6.bn3.running_var", "backbone.layers.2.7.conv1.weight", "backbone.layers.2.7.bn1.weight", "backbone.layers.2.7.bn1.bias", "backbone.layers.2.7.bn1.running_mean", "backbone.layers.2.7.bn1.running_var", "backbone.layers.2.7.conv2.weight", "backbone.layers.2.7.bn2.weight", "backbone.layers.2.7.bn2.bias", "backbone.layers.2.7.bn2.running_mean", "backbone.layers.2.7.bn2.running_var", "backbo